In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly import graph_objects as go
import plotly.io as pio
import json
from plotly.offline import plot
from WindPy import w
import datetime

In [2]:
w.start()
print(w.isconnected())

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
True


In [3]:
data = pd.read_csv("./index_data.csv")
data['date'] = pd.to_datetime(data['date'])

In [4]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,WASI_PB
0,2009-12-01,3560.8310,4394.9900,4627.0835,4109.1788,2901.8079,3.2825,NaN,NaN,NaN,3.5214
1,2009-12-02,3597.3290,4474.9260,4666.2974,4176.4685,2934.1360,3.3154,NaN,NaN,NaN,3.5602
2,2009-12-03,3590.8760,4515.8350,4638.4644,4212.0841,2932.4329,3.3019,NaN,NaN,NaN,3.5557
3,2009-12-04,3643.4910,4375.9790,4789.9594,4108.7244,2950.3362,3.3776,NaN,NaN,NaN,3.5971
4,2009-12-07,3668.8320,4459.0790,4786.9781,4182.6524,2972.4902,3.3856,NaN,NaN,NaN,3.6160
...,...,...,...,...,...,...,...,...,...,...,...
3842,2025-09-22,4522.6080,7489.4784,8020.9115,6036.1283,6241.6844,1.4514,2.5138,0.8890,3.1947,1.8032
3843,2025-09-23,4519.7781,7408.0691,8072.0016,6033.6812,6202.5976,1.4556,2.4894,0.8998,3.1888,1.7976
3844,2025-09-24,4566.0708,7534.2223,8074.9849,6194.5181,6289.6762,1.4645,2.5297,0.8992,3.2684,1.8169
3845,2025-09-25,4593.4875,7506.5124,8029.1614,6191.4151,6300.6440,1.4684,2.5200,0.8954,3.2634,1.8179


In [5]:
def previous_trading_day(date):
    # If today is Monday, move to Friday
    if date.weekday() == 0:
        return date - datetime.timedelta(days=3)
    # If today is Sunday, move to Friday
    elif date.weekday() == 6:
        return date - datetime.timedelta(days=2)
    # For weekdays (Monday to Friday), just go back one day
    else:
        return date - datetime.timedelta(days=1)

In [6]:
today = datetime.date.today().strftime("%Y-%m-%d")
last_trading_day = previous_trading_day(datetime.date.today())

In [7]:
# newdata = w.wss("000300.SH,000852.SH,399373.SZ,399376.SZ,881001.WI", "close,pb_lf",f"tradeDate={last_trading_day};priceAdj=U;cycle=D").Data
# newdata_list = newdata[0] + newdata[1]
# newdata_list = [round(x, 4) for x in newdata_list]
# data.iloc[len(data)-1,1:] = newdata_list
# data.loc[len(data), 'date'] = today

In [8]:
# data.to_csv("./index_data.csv", index = False)

#### **动量因子风格择时信号：**

##### 需要数据： 沪深300指数和中证1000指数日收盘价 
##### 信号逻辑： 取前第一日和第十一日指数收盘价计算过去十天指数收益率：
#####        (1) 沪深300指数收益率 > 中证1000指数收益率： 当天开盘买入沪深300指数,收盘平仓
#####        (2) 沪深300指数收益率 < 中证1000指数收益率： 当天开盘买入中证1000指数，收盘平仓
#####        (3) 若两者收益率均小于0：当天空仓

In [9]:

data["CSI300_Close_1"] = data['CSI300_Close'].shift()
data["CSI300_Close_11"] = data['CSI300_Close'].shift(11)
data["CSI1000_Close_1"] = data['CSI1000_Close'].shift()
data["CSI1000_Close_11"] = data['CSI1000_Close'].shift(11)
data['10_days_CSI300_rtn'] = data['CSI300_Close_1']/data['CSI300_Close_11']-1
data['10_days_CSI1000_rtn'] = data['CSI1000_Close_1']/data['CSI1000_Close_11']-1

In [10]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,WASI_PB,CSI300_Close_1,CSI300_Close_11,CSI1000_Close_1,CSI1000_Close_11,10_days_CSI300_rtn,10_days_CSI1000_rtn
0,2009-12-01,3560.8310,4394.9900,4627.0835,4109.1788,2901.8079,3.2825,NaN,NaN,NaN,3.5214,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-12-02,3597.3290,4474.9260,4666.2974,4176.4685,2934.1360,3.3154,NaN,NaN,NaN,3.5602,3560.8310,NaN,4394.9900,NaN,NaN,NaN
2,2009-12-03,3590.8760,4515.8350,4638.4644,4212.0841,2932.4329,3.3019,NaN,NaN,NaN,3.5557,3597.3290,NaN,4474.9260,NaN,NaN,NaN
3,2009-12-04,3643.4910,4375.9790,4789.9594,4108.7244,2950.3362,3.3776,NaN,NaN,NaN,3.5971,3590.8760,NaN,4515.8350,NaN,NaN,NaN
4,2009-12-07,3668.8320,4459.0790,4786.9781,4182.6524,2972.4902,3.3856,NaN,NaN,NaN,3.6160,3643.4910,NaN,4375.9790,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3842,2025-09-22,4522.6080,7489.4784,8020.9115,6036.1283,6241.6844,1.4514,2.5138,0.8890,3.1947,1.8032,4501.9195,4460.3249,7438.1887,7245.6670,0.009325,0.026571
3843,2025-09-23,4519.7781,7408.0691,8072.0016,6033.6812,6202.5976,1.4556,2.4894,0.8998,3.1888,1.7976,4522.6080,4467.5733,7489.4784,7311.0343,0.012319,0.024408
3844,2025-09-24,4566.0708,7534.2223,8074.9849,6194.5181,6289.6762,1.4645,2.5297,0.8992,3.2684,1.8169,4519.7781,4436.2584,7408.0691,7226.0277,0.018827,0.025192
3845,2025-09-25,4593.4875,7506.5124,8029.1614,6191.4151,6300.6440,1.4684,2.5200,0.8954,3.2634,1.8179,4566.0708,4445.3649,7534.2223,7230.1678,0.027153,0.042054


In [11]:
def signal1_row(row):
    if (row['10_days_CSI300_rtn'] * row['10_days_CSI1000_rtn'] > 0) & (row['10_days_CSI300_rtn'] < 0):
        row['Signal1'] = 0
        row['Signal1_day_rtn'] = 1
        row['Position1'] = '空仓'
    elif row['10_days_CSI300_rtn'] > row['10_days_CSI1000_rtn']:
        row['Signal1'] = 1
        row['Signal1_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position1'] = '满仓沪深300指数'
    else: 
        row['Signal1'] = -1
        row['Signal1_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position1'] = '满仓中证1000指数'
    return row

data = data.apply(signal1_row, axis = 1)

In [12]:
# fig1 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal1_rtn'],
#                  text = data['Position1'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig1.update_layout(template='simple_white',
#                    title='<b>动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig1.show()

#### **拥挤度动量因子风格择时信号：**

##### 需要数据： 大盘价值指数PB,小盘成长指数PB，万得全A指数PB 
##### 信号逻辑： 取前第一日和第五日指数PB计算指标：
#####           R1 = (大盘价值指数PB-小盘成长指数PB)/万得全A指数PB
#####        (1) R1(t-1) > R1(t-5)： 当天开盘买入沪深300指数,收盘平仓
#####        (2) R1(t-1) < R1(t-5)： 当天开盘买入中证1000指数，收盘平仓

In [13]:
data['LCVI_PB_1'] = data['LCVI_PB'].shift()
data['LCVI_PB_5'] = data['LCVI_PB'].shift(5)
data['SCGI_PB_1'] = data['SCGI_PB'].shift()
data['SCGI_PB_5'] = data['SCGI_PB'].shift(5)
data['WASI_PB_1'] = data['WASI_PB'].shift()
data['WASI_PB_5'] = data['WASI_PB'].shift(5)

In [14]:
def signal2_row(row):
    if (row['LCVI_PB_1'] - row['SCGI_PB_1'])/row['WASI_PB_1'] > (row['LCVI_PB_5'] - row['SCGI_PB_5'])/row['WASI_PB_5']:
        row['Signal2'] = 1
        row['Signal2_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position2'] = '满仓沪深300指数'
    else: 
        row['Signal2'] = -1
        row['Signal2_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position2'] = '满仓中证1000指数'
    return row

data = data.apply(signal2_row, axis = 1)

In [15]:
# fig2 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal2_rtn'],
#                  text = data['Position2'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig2.update_layout(template='simple_white',
#                    title='<b>拥挤度动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081",
#                    ),
#                    width=2000, 
#                    height=500)
# fig2.show()

#### **相对强弱动量因子风格择时信号（大盘价值指数和小盘成长指数）**

##### 需要数据： 大盘价值指数、小盘成长指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 大盘价值指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 小盘成长指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [16]:
data['LCVI_Close_1'] = data['LCVI_Close'].shift()
data['LCVI_Close_2'] = data['LCVI_Close'].shift(2)
data['SCGI_Close_1'] = data['SCGI_Close'].shift()
data['SCGI_Close_2'] = data['SCGI_Close'].shift(2)
data['WASI_Close_1'] = data['WASI_Close'].shift()
data['WASI_Close_2'] = data['WASI_Close'].shift(2)
data['LCVI_rtn_1'] = data['LCVI_Close_1']/data['LCVI_Close_2']
data['SCGI_rtn_1'] = data['SCGI_Close_1']/data['SCGI_Close_2']
data['WASI_rtn_1'] = data['WASI_Close_1']/data['WASI_Close_2']

In [17]:
def signal3_row(row):
    if (row['LCVI_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['SCGI_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal3'] = 0
        row['Signal3_day_rtn'] = 1
        row['Position3'] = '空仓'
    elif row['LCVI_rtn_1'] > row['SCGI_rtn_1']:
        row['Signal3'] = 1
        row['Signal3_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position3'] = '满仓沪深300指数'
    else: 
        row['Signal3'] = -1
        row['Signal3_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position3'] = '满仓中证1000指数'
    return row

data = data.apply(signal3_row, axis = 1)

In [18]:
# fig3 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal3_rtn'],
#                  text = data['Position3'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig3.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(大盘价值指数和小盘成长指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig3.show()

#### **相对强弱动量因子风格择时信号（沪深300指数和中证1000指数）**

##### 需要数据： 沪深300指数、中证1000指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 沪深300指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 中证1000指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [19]:
data['CSI300_Close_2'] = data['CSI300_Close'].shift(2)
data['CSI1000_Close_2'] = data['CSI1000_Close'].shift(2)
data['CSI300_rtn_1'] = data['CSI300_Close_1']/data['CSI300_Close_2']
data['CSI1000_rtn_1'] = data['CSI1000_Close_1']/data['CSI1000_Close_2']

In [20]:
def signal4_row(row):
    if (row['CSI300_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['CSI1000_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal4'] = 0
        row['Signal4_day_rtn'] = 1
        row['Position4'] = '空仓'
    elif row['CSI300_rtn_1'] > row['CSI1000_rtn_1']:
        row['Signal4'] = 1
        row['Signal4_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position4'] = '满仓沪深300指数'
    else: 
        row['Signal4'] = -1
        row['Signal4_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position4'] = '满仓中证1000指数'
    return row

data = data.apply(signal4_row, axis = 1)

In [21]:
# fig4 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal4_rtn'],
#                  text = data['Position4'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig4.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(沪深300指数和中证1000指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig4.show()

#### **多维动量复合风格择时对冲信号**

##### 信号逻辑： 根据以上四个信号计算指标
#####          c1 = 择时沪深300指数信号数量
#####          c2 = 择时中证1000指数信号数量
#####         (1) c1 - c2 > 2: 7.5%多IF + 7.5%空IM
#####         (2) 0 < c1 - c2 < 2: 5%多IF + 5%空IM
#####         (3) c1 - c2 = 0: 空仓
#####         (4) -2 < c1 - c2 < 0: 5%多IM + 5%空IF
#####         (5) c1 - c2 < -2: 7.5%多IM + 7.5%空IF

In [22]:
margin = 0.10
data['Signal_all'] = data['Signal1'] + data['Signal2'] + data['Signal3'] + data['Signal4']

In [23]:
def signal5_row(row):
    if (row['Signal_all'] > 2):
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IF + 7.5%空IM'
    elif row['Signal_all'] > 0:
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IF + 5%空IM'
    elif row['Signal_all'] == 0:
        row['Signal_all_day_rtn'] = 1
        row['Position5'] = '空仓'
    elif row['Signal_all'] > -2:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IM + 5%空IF'
    else:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IM + 7.5%空IF'
    return row

data = data.apply(signal5_row, axis = 1)

In [24]:
# fig5 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal5_rtn'],
#                  text = data['Position5'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig5.update_layout(template='simple_white',
#                    title='<b>多维动量复合风格择时对冲信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig5.show()

In [25]:
data = data.loc[data['date'] > '2010-01-01',:].reset_index(drop=True)

In [26]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,...,CSI300_Close_2,CSI1000_Close_2,CSI300_rtn_1,CSI1000_rtn_1,Signal4,Signal4_day_rtn,Position4,Signal_all,Signal_all_day_rtn,Position5
0,2010-01-04,3535.2290,4406.7650,4801.7380,4111.9040,2917.8525,3.3084,NaN,2.7787,4.3815,...,3558.8570,4336.6050,1.004728,1.012608,-1,1.003526,满仓中证1000指数,-4,1.011130,7.5%多IM + 7.5%空IF
1,2010-01-05,3564.0380,4462.2320,4829.1060,4158.2700,2941.4729,3.3449,NaN,2.8017,4.4299,...,3575.6840,4391.2810,0.988686,1.003526,-1,1.012587,满仓中证1000指数,-4,1.003328,7.5%多IM + 7.5%空IF
2,2010-01-06,3541.7270,4452.8680,4780.3700,4153.2210,2921.8748,3.3133,NaN,2.7692,4.3964,...,3535.2290,4406.7650,1.008149,1.012587,-1,0.997901,满仓中证1000指数,-4,1.003121,7.5%多IM + 7.5%空IF
3,2010-01-07,3471.4560,4361.1610,4673.6460,4072.9810,2860.3718,3.2496,NaN,2.7060,4.3115,...,3564.0380,4462.2320,0.993740,0.997901,-1,0.979405,满仓中证1000指数,-4,0.999434,7.5%多IM + 7.5%空IF
4,2010-01-08,3480.1300,4417.7710,4683.2360,4126.6380,2872.4092,3.2475,NaN,2.7084,4.3647,...,3541.7270,4452.8680,0.980159,0.979405,0,1.000000,空仓,-2,1.007861,7.5%多IM + 7.5%空IF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3819,2025-09-22,4522.6080,7489.4784,8020.9115,6036.1283,6241.6844,1.4514,2.5138,0.8890,3.1947,...,4498.1090,7476.4020,1.000847,0.994889,1,1.004595,满仓沪深300指数,0,1.000000,空仓
3820,2025-09-23,4519.7781,7408.0691,8072.0016,6033.6812,6202.5976,1.4556,2.4894,0.8998,3.1888,...,4501.9195,7438.1887,1.004595,1.006895,0,1.000000,空仓,-3,0.992317,7.5%多IM + 7.5%空IF
3821,2025-09-24,4566.0708,7534.2223,8074.9849,6194.5181,6289.6762,1.4645,2.5297,0.8992,3.2684,...,4522.6080,7489.4784,0.999374,0.989130,1,1.010242,满仓沪深300指数,0,1.000000,空仓
3822,2025-09-25,4593.4875,7506.5124,8029.1614,6191.4151,6300.6440,1.4684,2.5200,0.8954,3.2634,...,4519.7781,7408.0691,1.010242,1.017029,-1,0.996322,满仓中证1000指数,-4,0.992738,7.5%多IM + 7.5%空IF


In [27]:
data.iloc[-10:,[0,-1]].reset_index(drop = True)

,date,Position5
0,2025-09-15,7.5%多IM + 7.5%空IF
1,2025-09-16,空仓
2,2025-09-17,7.5%多IM + 7.5%空IF
3,2025-09-18,7.5%多IM + 7.5%空IF
4,2025-09-19,7.5%多IM + 7.5%空IF
5,2025-09-22,空仓
6,2025-09-23,7.5%多IM + 7.5%空IF
7,2025-09-24,空仓
8,2025-09-25,7.5%多IM + 7.5%空IF
9,2025-09-26,5%多IM + 5%空IF


In [28]:
data['year'] = data['date'].dt.year

In [29]:
data['Signal1_Value'] = data.groupby('year')['Signal1_day_rtn'].cumprod() * 500000
data['Signal2_Value'] = data.groupby('year')['Signal2_day_rtn'].cumprod() * 500000
data['Signal3_Value'] = data.groupby('year')['Signal3_day_rtn'].cumprod() * 500000
data['Signal4_Value'] = data.groupby('year')['Signal4_day_rtn'].cumprod() * 500000
data['Signal5_Value'] = data.groupby('year')['Signal_all_day_rtn'].cumprod() * 500000

In [30]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,...,Position4,Signal_all,Signal_all_day_rtn,Position5,year,Signal1_Value,Signal2_Value,Signal3_Value,Signal4_Value,Signal5_Value
0,2010-01-04,3535.2290,4406.7650,4801.7380,4111.9040,2917.8525,3.3084,NaN,2.7787,4.3815,...,满仓中证1000指数,-4,1.011130,7.5%多IM + 7.5%空IF,2010,501763.039077,501763.039077,501763.039077,501763.039077,505564.999022
1,2010-01-05,3564.0380,4462.2320,4829.1060,4158.2700,2941.4729,3.3449,NaN,2.8017,4.4299,...,满仓中证1000指数,-4,1.003328,7.5%多IM + 7.5%空IF,2010,508078.622161,508078.622161,508078.622161,508078.622161,507247.645362
2,2010-01-06,3541.7270,4452.8680,4780.3700,4153.2210,2921.8748,3.3133,NaN,2.7692,4.3964,...,满仓中证1000指数,-4,1.003121,7.5%多IM + 7.5%空IF,2010,507012.418472,507012.418472,507012.418472,507012.418472,508830.841180
3,2010-01-07,3471.4560,4361.1610,4673.6460,4072.9810,2860.3718,3.2496,NaN,2.7060,4.3115,...,满仓中证1000指数,-4,0.999434,7.5%多IM + 7.5%空IF,2010,496570.476815,496570.476815,496570.476815,496570.476815,508543.037927
4,2010-01-08,3480.1300,4417.7710,4683.2360,4126.6380,2872.4092,3.2475,NaN,2.7084,4.3647,...,空仓,-2,1.007861,7.5%多IM + 7.5%空IF,2010,503016.204155,503016.204155,496570.476815,496570.476815,512540.882299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3819,2025-09-22,4522.6080,7489.4784,8020.9115,6036.1283,6241.6844,1.4514,2.5138,0.8890,3.1947,...,满仓沪深300指数,0,1.000000,空仓,2025,609678.884045,591914.422861,567930.914610,647365.345494,505741.807663
3820,2025-09-23,4519.7781,7408.0691,8072.0016,6033.6812,6202.5976,1.4556,2.4894,0.8998,3.1888,...,空仓,-3,0.992317,7.5%多IM + 7.5%空IF,2025,603051.782861,585480.418215,561757.606439,647365.345494,501856.155669
3821,2025-09-24,4566.0708,7534.2223,8074.9849,6194.5181,6289.6762,1.4645,2.5297,0.8992,3.2684,...,满仓沪深300指数,0,1.000000,空仓,2025,613321.248648,595450.658408,567511.268626,653995.823599,501856.155669
3822,2025-09-25,4593.4875,7506.5124,8029.1614,6191.4151,6300.6440,1.4684,2.5200,0.8954,3.2634,...,满仓中证1000指数,-4,0.992738,7.5%多IM + 7.5%空IF,2025,611065.532027,593260.667518,565424.035216,651590.511126,498211.809869


In [31]:
output = data[['date','Signal1_Value','Position1','Signal2_Value','Position2', 'Signal3_Value','Position3', 'Signal4_Value','Position4', 'Signal5_Value','Position5']]

In [32]:
output

,date,Signal1_Value,Position1,Signal2_Value,Position2,Signal3_Value,Position3,Signal4_Value,Position4,Signal5_Value,Position5
0,2010-01-04,501763.039077,满仓中证1000指数,501763.039077,满仓中证1000指数,501763.039077,满仓中证1000指数,501763.039077,满仓中证1000指数,505564.999022,7.5%多IM + 7.5%空IF
1,2010-01-05,508078.622161,满仓中证1000指数,508078.622161,满仓中证1000指数,508078.622161,满仓中证1000指数,508078.622161,满仓中证1000指数,507247.645362,7.5%多IM + 7.5%空IF
2,2010-01-06,507012.418472,满仓中证1000指数,507012.418472,满仓中证1000指数,507012.418472,满仓中证1000指数,507012.418472,满仓中证1000指数,508830.841180,7.5%多IM + 7.5%空IF
3,2010-01-07,496570.476815,满仓中证1000指数,496570.476815,满仓中证1000指数,496570.476815,满仓中证1000指数,496570.476815,满仓中证1000指数,508543.037927,7.5%多IM + 7.5%空IF
4,2010-01-08,503016.204155,满仓中证1000指数,503016.204155,满仓中证1000指数,496570.476815,空仓,496570.476815,空仓,512540.882299,7.5%多IM + 7.5%空IF
...,...,...,...,...,...,...,...,...,...,...,...
3819,2025-09-22,609678.884045,满仓中证1000指数,591914.422861,满仓中证1000指数,567930.914610,满仓沪深300指数,647365.345494,满仓沪深300指数,505741.807663,空仓
3820,2025-09-23,603051.782861,满仓中证1000指数,585480.418215,满仓中证1000指数,561757.606439,满仓中证1000指数,647365.345494,空仓,501856.155669,7.5%多IM + 7.5%空IF
3821,2025-09-24,613321.248648,满仓中证1000指数,595450.658408,满仓中证1000指数,567511.268626,满仓沪深300指数,653995.823599,满仓沪深300指数,501856.155669,空仓
3822,2025-09-25,611065.532027,满仓中证1000指数,593260.667518,满仓中证1000指数,565424.035216,满仓中证1000指数,651590.511126,满仓中证1000指数,498211.809869,7.5%多IM + 7.5%空IF


In [34]:
output.to_csv('./output.csv')

In [33]:
# data['Signal1_Value'] = np.cumprod(data['Signal1_day_rtn']) * 500000
# data['Signal2_Value'] = np.cumprod(data['Signal2_day_rtn']) * 500000
# data['Signal3_Value'] = np.cumprod(data['Signal3_day_rtn']) * 500000
# data['Signal4_Value'] = np.cumprod(data['Signal4_day_rtn']) * 500000
# data['Signal5_Value'] = np.cumprod(data['Signal_all_day_rtn']) * 500000